In [1]:
import pandas as pd
import numpy as np
import os
import time
import random
import pprint
import yaml
import warnings
warnings.filterwarnings(action="ignore")


import pandas as pd
import numpy as np
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import KFold
import random
toacco_geo = pd.read_csv("feature/toacco_geo2_mit_order.csv")
toacco_geo_chem = pd.read_csv("feature/toacco_geo_chem_erase_mit_order.csv")
toacco_geo = toacco_geo.iloc[:,2:]
toacco_geo.head()

,name,topology,First nodular symmetry code,First nodular character,First nodular ID,Second nodular symmetry code,Second nodular character,Second nodular ID,Connecting building block ID,Di,...,VPOV,GPOV,POAV_vol_frac,PONAV_vol_frac,GPOAV,GPONAV,POAV,PONAV,bulk,shear
0,acs_sym_7_mc_4_L_12,acs,7,metaliic,4,7,metallic,4,12,9.64336,...,0.6635,0.946242,0.6635,0.0,0.946242,0.0,2087.70,0.0,14.128650,11.231780
1,acs_sym_7_mc_4_L_13,acs,7,metaliic,4,7,metallic,4,13,9.94957,...,0.7011,0.953612,0.7011,0.0,0.953611,0.0,2141.51,0.0,14.759740,11.627400
2,acs_sym_7_mc_4_L_20,acs,7,metaliic,4,7,metallic,4,20,13.42609,...,0.7960,1.846966,0.7960,0.0,1.846970,0.0,4517.02,0.0,8.911607,5.467771
3,acs_sym_7_mc_4_L_24,acs,7,metaliic,4,7,metallic,4,24,15.06601,...,0.8043,2.149081,0.8043,0.0,2.149080,0.0,6370.94,0.0,6.867950,4.384087
4,acs_sym_7_mc_4_L_26,acs,7,metaliic,4,7,metallic,4,26,15.35307,...,0.8313,2.061613,0.8313,0.0,2.061610,0.0,6273.98,0.0,7.366024,5.053149


In [2]:
geometric = ['Di', 'Df', 'Dif', 'rho', 'VSA','GSA', 'VPOV', 'GPOV', 'POAV_vol_frac', 'PONAV_vol_frac', 'GPOAV','GPONAV', 'POAV', 'PONAV']
RAC = ['f-chi-0-all', 'f-chi-1-all', 'f-chi-2-all',
       'f-chi-3-all', 'f-Z-0-all', 'f-Z-1-all', 'f-Z-2-all', 'f-Z-3-all',
       'f-I-0-all', 'f-I-1-all', 'f-I-2-all', 'f-I-3-all', 'f-T-0-all',
       'f-T-1-all', 'f-T-2-all', 'f-T-3-all', 'f-S-0-all', 'f-S-1-all',
       'f-S-2-all', 'f-S-3-all', 'mc-chi-0-all', 'mc-chi-1-all',
       'mc-chi-2-all', 'mc-chi-3-all', 'mc-Z-0-all', 'mc-Z-1-all',
       'mc-Z-2-all', 'mc-Z-3-all', 'mc-I-0-all', 'mc-I-1-all', 'mc-I-2-all',
       'mc-I-3-all', 'mc-T-0-all', 'mc-T-1-all', 'mc-T-2-all', 'mc-T-3-all',
       'mc-S-0-all', 'mc-S-1-all', 'mc-S-2-all', 'mc-S-3-all',
       'D_mc-chi-0-all', 'D_mc-chi-1-all', 'D_mc-chi-2-all', 'D_mc-chi-3-all',
       'D_mc-Z-0-all', 'D_mc-Z-1-all', 'D_mc-Z-2-all', 'D_mc-Z-3-all',
       'D_mc-I-0-all', 'D_mc-I-1-all', 'D_mc-I-2-all', 'D_mc-I-3-all',
       'D_mc-T-0-all', 'D_mc-T-1-all', 'D_mc-T-2-all', 'D_mc-T-3-all',
       'D_mc-S-0-all', 'D_mc-S-1-all', 'D_mc-S-2-all', 'D_mc-S-3-all',
       'f-lig-chi-0', 'f-lig-chi-1', 'f-lig-chi-2', 'f-lig-chi-3', 'f-lig-Z-0',
       'f-lig-Z-1', 'f-lig-Z-2', 'f-lig-Z-3', 'f-lig-I-0', 'f-lig-I-1',
       'f-lig-I-2', 'f-lig-I-3', 'f-lig-T-0', 'f-lig-T-1', 'f-lig-T-2',
       'f-lig-T-3', 'f-lig-S-0', 'f-lig-S-1', 'f-lig-S-2', 'f-lig-S-3',
       'lc-chi-0-all', 'lc-chi-1-all', 'lc-chi-2-all', 'lc-chi-3-all',
       'lc-Z-0-all', 'lc-Z-1-all', 'lc-Z-2-all', 'lc-Z-3-all', 'lc-I-0-all',
       'lc-I-1-all', 'lc-I-2-all', 'lc-I-3-all', 'lc-T-0-all', 'lc-T-1-all',
       'lc-T-2-all', 'lc-T-3-all', 'lc-S-0-all', 'lc-S-1-all', 'lc-S-2-all',
        'lc-S-3-all', 'lc-alpha-0-all', 'lc-alpha-1-all', 'lc-alpha-2-all',
       'lc-alpha-3-all', 'D_lc-chi-0-all', 'D_lc-chi-1-all', 'D_lc-chi-2-all',
       'D_lc-chi-3-all', 'D_lc-Z-0-all', 'D_lc-Z-1-all', 'D_lc-Z-2-all',
       'D_lc-Z-3-all', 'D_lc-I-0-all', 'D_lc-I-1-all', 'D_lc-I-2-all',
       'D_lc-I-3-all', 'D_lc-T-0-all', 'D_lc-T-1-all', 'D_lc-T-2-all',
       'D_lc-T-3-all', 'D_lc-S-0-all', 'D_lc-S-1-all', 'D_lc-S-2-all',
       'D_lc-S-3-all', 'D_lc-alpha-0-all', 'D_lc-alpha-1-all',
       'D_lc-alpha-2-all', 'D_lc-alpha-3-all', 'func-chi-0-all',
       'func-chi-1-all', 'func-chi-2-all', 'func-chi-3-all', 'func-Z-0-all',
       'func-Z-1-all', 'func-Z-2-all', 'func-Z-3-all', 'func-I-0-all',
       'func-I-1-all', 'func-I-2-all', 'func-I-3-all', 'func-T-0-all',
       'func-T-1-all', 'func-T-2-all', 'func-T-3-all', 'func-S-0-all',
       'func-S-1-all', 'func-S-2-all', 'func-S-3-all', 'func-alpha-0-all',
       'func-alpha-1-all', 'func-alpha-2-all', 'func-alpha-3-all',
       'D_func-chi-0-all', 'D_func-chi-1-all', 'D_func-chi-2-all',
       'D_func-chi-3-all', 'D_func-Z-0-all', 'D_func-Z-1-all',
       'D_func-Z-2-all', 'D_func-Z-3-all', 'D_func-I-0-all', 'D_func-I-1-all',
       'D_func-I-2-all', 'D_func-I-3-all', 'D_func-T-0-all', 'D_func-T-1-all',
       'D_func-T-2-all', 'D_func-T-3-all', 'D_func-S-0-all', 'D_func-S-1-all',
       'D_func-S-2-all', 'D_func-S-3-all', 'D_func-alpha-0-all',
       'D_func-alpha-1-all', 'D_func-alpha-2-all', 'D_func-alpha-3-all']
property_ = ["bulk"]
## define the domain
geo = geometric + property_
RAC_geo = geometric + RAC + property_

In [3]:
X = np.array(toacco_geo_chem[geometric])
y = np.array(toacco_geo_chem[property_])

In [4]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

loss_history = []
accuracy = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = ExtraTreesRegressor(random_state=42) 
    model.fit(X_train, y_train) 

    y_pred = model.predict(X_test) 
    loss_history.append(mean_squared_error(y_test, y_pred,squared=False))
    accuracy.append(r2_score(y_test,y_pred,))
print("history of rmse :", loss_history)
print("average rmse value  :", np.mean(loss_history))
print("history of r2 square :",accuracy)
print("averare r2 square :" ,np.mean(accuracy))

history of rmse : [2.8876123289837663, 2.29045028007943, 3.1349846444823615, 2.629421976053353, 3.302039809721424, 3.294325886695139, 2.7296087691893716, 2.599568667258422, 2.7161930533312995, 2.161362897167933]
average rmse value  : 2.77455683129625
history of r2 square : [0.8230678893295891, 0.8449554862368197, 0.798738261774036, 0.827201389390473, 0.8468967583264814, 0.8317520932481757, 0.7466981582531529, 0.8604456745899336, 0.8046125156983737, 0.8673930525393615]
averare r2 square : 0.8251761279386397


In [5]:
import optuna
from sklearn import datasets

def objective(trial,X=X, y=y):
    
        
    n_estimators = trial.suggest_int("n_estimators", 50, 500, 50)
        
    max_features = trial.suggest_categorical("max_features",["sqrt","log2"])
        
    max_depth = trial.suggest_categorical("max_depth",[None,10,20,30,40,50,60,70,80,90,100])
        
    min_samples_split = trial.suggest_int("min_samples_split",2,10,2)
        
    min_samples_leaf = trial.suggest_int("min_samples_leaf",1,5,1)
        
    bootstrap = trial.suggest_categorical("bootstrap",[True,False])
    
    criterion = trial.suggest_categorical("criterion",["mse","mae"])

    accuracies = []
    
    kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

    loss = []

    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = ExtraTreesRegressor(random_state=42,
                                      n_estimators=n_estimators,
                                      max_features = max_features,
                                      max_depth = max_depth,
                                      min_samples_split = min_samples_split,
                                      min_samples_leaf = min_samples_leaf,
                                      bootstrap = bootstrap,
                                      criterion = criterion) 
        model.fit(X_train, y_train) 

        y_pred = model.predict(X_test) 
        
        rmse = mean_squared_error(y_test,y_pred,squared = False)
        
        loss.append(rmse)
        average_loss = np.mean(loss)
        
    return average_loss
    
    
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials = 160)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print("best rmse is :",study.best_trial.values)

[I 2022-11-24 16:14:04,076] A new study created in memory with name: no-name-36081288-9488-4e8f-ae8e-f54c3c4be8d1
[I 2022-11-24 16:14:48,616] Trial 0 finished with value: 3.3652563089325165 and parameters: {'n_estimators': 300, 'max_features': 'sqrt', 'max_depth': 50, 'min_samples_split': 8, 'min_samples_leaf': 2, 'bootstrap': True, 'criterion': 'mae'}. Best is trial 0 with value: 3.3652563089325165.
[I 2022-11-24 16:14:50,437] Trial 1 finished with value: 3.426996677522189 and parameters: {'n_estimators': 150, 'max_features': 'sqrt', 'max_depth': 10, 'min_samples_split': 8, 'min_samples_leaf': 1, 'bootstrap': True, 'criterion': 'mse'}. Best is trial 0 with value: 3.3652563089325165.
[I 2022-11-24 16:14:55,902] Trial 2 finished with value: 3.3384858880760127 and parameters: {'n_estimators': 500, 'max_features': 'sqrt', 'max_depth': 10, 'min_samples_split': 6, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 2 with value: 3.3384858880760127.
[I 2022-11-24 16

[I 2022-11-24 16:37:51,502] Trial 27 finished with value: 3.4067717613272857 and parameters: {'n_estimators': 100, 'max_features': 'sqrt', 'max_depth': None, 'min_samples_split': 4, 'min_samples_leaf': 4, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 24 with value: 2.9359775457754145.
[I 2022-11-24 16:38:24,860] Trial 28 finished with value: 3.1389025522772944 and parameters: {'n_estimators': 100, 'max_features': 'sqrt', 'max_depth': 40, 'min_samples_split': 2, 'min_samples_leaf': 2, 'bootstrap': False, 'criterion': 'mae'}. Best is trial 24 with value: 2.9359775457754145.
[I 2022-11-24 16:38:51,321] Trial 29 finished with value: 3.0950216317452224 and parameters: {'n_estimators': 150, 'max_features': 'sqrt', 'max_depth': 100, 'min_samples_split': 4, 'min_samples_leaf': 1, 'bootstrap': True, 'criterion': 'mae'}. Best is trial 24 with value: 2.9359775457754145.
[I 2022-11-24 16:39:22,106] Trial 30 finished with value: 3.2955604528183735 and parameters: {'n_estimators': 100, 'max

[I 2022-11-24 16:52:33,824] Trial 54 finished with value: 2.8370989029585725 and parameters: {'n_estimators': 500, 'max_features': 'log2', 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 44 with value: 2.8370989029585725.
[I 2022-11-24 16:52:40,732] Trial 55 finished with value: 3.22369283442172 and parameters: {'n_estimators': 500, 'max_features': 'log2', 'max_depth': 30, 'min_samples_split': 10, 'min_samples_leaf': 2, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 44 with value: 2.8370989029585725.
[I 2022-11-24 16:52:52,829] Trial 56 finished with value: 2.8399513224292985 and parameters: {'n_estimators': 500, 'max_features': 'log2', 'max_depth': 80, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 44 with value: 2.8370989029585725.
[I 2022-11-24 16:53:03,723] Trial 57 finished with value: 2.837848172765425 and parameters: {'n_estimators': 450, 'max_fea

[I 2022-11-24 16:57:04,243] Trial 81 finished with value: 2.8370989029585725 and parameters: {'n_estimators': 500, 'max_features': 'log2', 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 44 with value: 2.8370989029585725.
[I 2022-11-24 16:57:16,350] Trial 82 finished with value: 2.8370989029585725 and parameters: {'n_estimators': 500, 'max_features': 'log2', 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 44 with value: 2.8370989029585725.
[I 2022-11-24 16:57:28,447] Trial 83 finished with value: 2.8370989029585725 and parameters: {'n_estimators': 500, 'max_features': 'log2', 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 44 with value: 2.8370989029585725.
[I 2022-11-24 16:57:40,549] Trial 84 finished with value: 2.8370989029585725 and parameters: {'n_estimators': 500, 'max_f

[I 2022-11-24 17:02:04,137] Trial 108 finished with value: 2.9288403197832134 and parameters: {'n_estimators': 500, 'max_features': 'log2', 'max_depth': 80, 'min_samples_split': 4, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 44 with value: 2.8370989029585725.
[I 2022-11-24 17:02:09,295] Trial 109 finished with value: 3.5114879152141127 and parameters: {'n_estimators': 450, 'max_features': 'log2', 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 5, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 44 with value: 2.8370989029585725.
[I 2022-11-24 17:02:16,635] Trial 110 finished with value: 3.1341221404886155 and parameters: {'n_estimators': 500, 'max_features': 'log2', 'max_depth': 20, 'min_samples_split': 10, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 44 with value: 2.8370989029585725.
[I 2022-11-24 17:02:28,732] Trial 111 finished with value: 2.8370989029585725 and parameters: {'n_estimators': 500, '

[I 2022-11-24 17:06:55,937] Trial 135 finished with value: 2.837848172765425 and parameters: {'n_estimators': 450, 'max_features': 'log2', 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 44 with value: 2.8370989029585725.
[I 2022-11-24 17:07:08,039] Trial 136 finished with value: 2.8370989029585725 and parameters: {'n_estimators': 500, 'max_features': 'log2', 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 44 with value: 2.8370989029585725.
[I 2022-11-24 17:07:09,066] Trial 137 finished with value: 2.9969583875833923 and parameters: {'n_estimators': 50, 'max_features': 'log2', 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': True, 'criterion': 'mse'}. Best is trial 44 with value: 2.8370989029585725.
[I 2022-11-24 17:07:20,538] Trial 138 finished with value: 2.8399513224292985 and parameters: {'n_estimators': 500, 'max_

Number of finished trials: 160
Best trial: {'n_estimators': 500, 'max_features': 'sqrt', 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mse'}
best rmse is : [2.8370989029585725]


In [7]:
params = study.best_trial.params

In [8]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

loss_history = []
accuracy = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = ExtraTreesRegressor(random_state=42,**params) 
    model.fit(X_train, y_train) 

    y_pred = model.predict(X_test) 
    loss_history.append(mean_squared_error(y_test, y_pred,squared=False))
    accuracy.append(r2_score(y_test,y_pred,))
print("history of rmse :", loss_history)
print("average rmse value  :", np.mean(loss_history))
print("history of r2 square :",accuracy)
print("averare r2 square :" ,np.mean(accuracy))

history of rmse : [3.1600472921850296, 2.3328140919289733, 3.1635420918192496, 2.627140921995901, 3.3771640559233123, 3.3868643912705014, 2.857143532664122, 2.4927560131495268, 2.720608973292093, 2.2529076653570157]
average rmse value  : 2.8370989029585725
history of r2 square : [0.7881072741653128, 0.8391670877040973, 0.7950548629499677, 0.8275010689435965, 0.8398510509315548, 0.8221670766499658, 0.7224752917456125, 0.8716782542403154, 0.8039766869136143, 0.8559219972739937]
averare r2 square : 0.8165900651518031


# Topology 전용

In [3]:
import pandas as pd
import numpy as np
import os
import time
import random
import pprint
import yaml
import warnings
warnings.filterwarnings(action="ignore")


import pandas as pd
import numpy as np
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import KFold
import random
toacco_geo = pd.read_csv("feature/toacco_geo2_mit_order.csv")
data = pd.read_csv("feature/toacco_geo_chem_erase_mit_order.csv")
data = data.iloc[:,2:]
data.head()

,MOF ID,name,topology,First nodular symmetry code,First nodular character,First nodular ID,Second nodular symmetry code,Second nodular character,Second nodular ID,Connecting building block ID,...,D_func-S-0-all,D_func-S-1-all,D_func-S-2-all,D_func-S-3-all,D_func-alpha-0-all,D_func-alpha-1-all,D_func-alpha-2-all,D_func-alpha-3-all,bulk,shear
0,3,acs_sym_7_mc_4_L_12,acs,7,metaliic,4,7,metallic,4,12,...,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,14.128650,11.231780
1,4,acs_sym_7_mc_4_L_13,acs,7,metaliic,4,7,metallic,4,13,...,0.0,-0.02,-0.04,0.02,0.0,-3.9,-7.8,0.3,14.759740,11.627400
2,12,acs_sym_7_mc_4_L_20,acs,7,metaliic,4,7,metallic,4,20,...,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,8.911607,5.467771
3,16,acs_sym_7_mc_4_L_24,acs,7,metaliic,4,7,metallic,4,24,...,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,6.867950,4.384087
4,18,acs_sym_7_mc_4_L_26,acs,7,metaliic,4,7,metallic,4,26,...,0.0,-0.02,-0.04,0.00,0.0,-3.9,-7.8,-1.8,7.366024,5.053149


In [4]:
from sklearn.preprocessing import OneHotEncoder
geometric = ["topology",'Di', 'Df', 'Dif', 'rho', 'VSA',
       'GSA', 'VPOV', 'GPOV', 'POAV_vol_frac', 'PONAV_vol_frac', 'GPOAV',
       'GPONAV', 'POAV', 'PONAV']
X = data[geometric]
y = data["bulk"]

onehotencoder = OneHotEncoder()
top_one = onehotencoder.fit_transform(X.topology.values.reshape(-1,1)).toarray()

df_top_one = pd.DataFrame(top_one,columns=["topology_"+str(int(i)) for i in range(40)])

df = pd.concat([X, df_top_one,y], axis=1)
df= df.drop(['topology'], axis=1) 
print(df.head())

         Di        Df       Dif      rho      VSA      GSA    VPOV      GPOV  \
0   9.64336   8.80746   9.64336  3146.50  2193.80  3128.66  0.6635  0.946242   
1   9.94957   9.48351   9.94957  3054.50  2236.08  3041.44  0.7011  0.953612   
2  13.42609  11.83384  13.42609  5674.65  2069.82  4802.63  0.7960  1.846966   
3  15.06601  14.00604  15.05182  7921.10  1838.19  4911.62  0.8043  2.149081   
4  15.35307  14.52062  15.35307  7547.20  1783.35  4422.69  0.8313  2.061613   

   POAV_vol_frac  PONAV_vol_frac  ...  topology_31  topology_32  topology_33  \
0         0.6635             0.0  ...          0.0          0.0          0.0   
1         0.7011             0.0  ...          0.0          0.0          0.0   
2         0.7960             0.0  ...          0.0          0.0          0.0   
3         0.8043             0.0  ...          0.0          0.0          0.0   
4         0.8313             0.0  ...          0.0          0.0          0.0   

   topology_34  topology_35  topology_

In [5]:
X = np.array(df.iloc[:,:-1])
y = np.array(df.iloc[:,-1])
# topology + geometric one hot encoding
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

loss_history = []
accuracy = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = ExtraTreesRegressor(random_state=42) 
    model.fit(X_train, y_train) 

    y_pred = model.predict(X_test) 
    loss_history.append(mean_squared_error(y_test, y_pred,squared=False))
    accuracy.append(r2_score(y_test,y_pred,))
print("history of rmse :", loss_history)
print("average rmse value  :", np.mean(loss_history))
print("history of r2 square :",accuracy)
print("averare r2 square :" ,np.mean(accuracy))

history of rmse : [2.5552010949997257, 2.0105755681287985, 2.630571714269483, 2.3476174267433874, 2.4711470331382626, 2.820061106691242, 2.3666736575284815, 2.176714742642817, 2.089791177949974, 1.907625244304391]
average rmse value  : 2.337597876639656
history of r2 square : [0.8614587654515797, 0.8805309342506558, 0.8582932051291188, 0.8622554835116238, 0.9142533918525011, 0.876708371302444, 0.809579241026691, 0.902153847811219, 0.8843405145673039, 0.8967007862224918]
averare r2 square : 0.8746274541125629


In [6]:
import optuna
from sklearn import datasets

def objective(trial,X=X, y=y):
    
        
    n_estimators = trial.suggest_int("n_estimators", 50, 500, 50)
        
    max_features = trial.suggest_categorical("max_features",["sqrt","log2"])
        
    max_depth = trial.suggest_categorical("max_depth",[None,10,20,30,40,50,60,70,80,90,100])
        
    min_samples_split = trial.suggest_int("min_samples_split",2,10,2)
        
    min_samples_leaf = trial.suggest_int("min_samples_leaf",1,5,1)
        
    bootstrap = trial.suggest_categorical("bootstrap",[True,False])
    
    criterion = trial.suggest_categorical("criterion",["mse","mae"])

    accuracies = []
    
    kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

    loss = []

    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = ExtraTreesRegressor(random_state=42,
                                      n_estimators=n_estimators,
                                      max_features = max_features,
                                      max_depth = max_depth,
                                      min_samples_split = min_samples_split,
                                      min_samples_leaf = min_samples_leaf,
                                      bootstrap = bootstrap,
                                      criterion = criterion) 
        model.fit(X_train, y_train) 

        y_pred = model.predict(X_test) 
        
        rmse = mean_squared_error(y_test,y_pred,squared = False)
        
        loss.append(rmse)
        average_loss = np.mean(loss)
        
    return average_loss
    
    
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials = 160)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print("best rmse is :",study.best_trial.values)

[I 2022-11-25 00:31:24,758] A new study created in memory with name: no-name-612bddd9-479a-48a4-b7bf-441919328680
[I 2022-11-25 00:32:16,002] Trial 0 finished with value: 3.1381317112751854 and parameters: {'n_estimators': 200, 'max_features': 'log2', 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 2, 'bootstrap': True, 'criterion': 'mae'}. Best is trial 0 with value: 3.1381317112751854.
[I 2022-11-25 00:36:14,270] Trial 1 finished with value: 2.762385445877529 and parameters: {'n_estimators': 400, 'max_features': 'log2', 'max_depth': 20, 'min_samples_split': 10, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mae'}. Best is trial 1 with value: 2.762385445877529.
[I 2022-11-25 00:37:53,738] Trial 2 finished with value: 2.7035216044687087 and parameters: {'n_estimators': 150, 'max_features': 'sqrt', 'max_depth': 60, 'min_samples_split': 4, 'min_samples_leaf': 2, 'bootstrap': False, 'criterion': 'mae'}. Best is trial 2 with value: 2.7035216044687087.
[I 2022-11-25 0

[I 2022-11-25 00:53:46,196] Trial 27 finished with value: 2.7408976917058525 and parameters: {'n_estimators': 250, 'max_features': 'sqrt', 'max_depth': 50, 'min_samples_split': 8, 'min_samples_leaf': 2, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 7 with value: 2.3084101011375617.
[I 2022-11-25 00:53:53,985] Trial 28 finished with value: 2.3840896180127875 and parameters: {'n_estimators': 350, 'max_features': 'sqrt', 'max_depth': 60, 'min_samples_split': 4, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 7 with value: 2.3084101011375617.
[I 2022-11-25 00:53:57,077] Trial 29 finished with value: 2.9893929291738113 and parameters: {'n_estimators': 200, 'max_features': 'log2', 'max_depth': 90, 'min_samples_split': 2, 'min_samples_leaf': 2, 'bootstrap': True, 'criterion': 'mse'}. Best is trial 7 with value: 2.3084101011375617.
[I 2022-11-25 00:54:00,224] Trial 30 finished with value: 2.470063202755483 and parameters: {'n_estimators': 150, 'max_featur

[I 2022-11-25 01:09:32,831] Trial 54 finished with value: 2.3083214502889997 and parameters: {'n_estimators': 250, 'max_features': 'sqrt', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 37 with value: 2.3083214502889997.
[I 2022-11-25 01:09:39,750] Trial 55 finished with value: 2.3083214502889997 and parameters: {'n_estimators': 250, 'max_features': 'sqrt', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 37 with value: 2.3083214502889997.
[I 2022-11-25 01:09:43,568] Trial 56 finished with value: 2.6840347066561376 and parameters: {'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': None, 'min_samples_split': 4, 'min_samples_leaf': 2, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 37 with value: 2.3083214502889997.
[I 2022-11-25 01:09:50,497] Trial 57 finished with value: 2.3083214502889997 and parameters: {'n_estimators': 250, 

[I 2022-11-25 01:16:11,194] Trial 81 finished with value: 2.30642945864066 and parameters: {'n_estimators': 350, 'max_features': 'sqrt', 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 72 with value: 2.30642945864066.
[I 2022-11-25 01:16:22,219] Trial 82 finished with value: 2.3095068415007414 and parameters: {'n_estimators': 400, 'max_features': 'sqrt', 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 72 with value: 2.30642945864066.
[I 2022-11-25 01:16:31,919] Trial 83 finished with value: 2.30642945864066 and parameters: {'n_estimators': 350, 'max_features': 'sqrt', 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mse'}. Best is trial 72 with value: 2.30642945864066.
[I 2022-11-25 01:16:41,565] Trial 84 finished with value: 2.30642945864066 and parameters: {'n_estimators': 350, 'max_features': 's

[I 2022-11-25 03:13:14,176] Trial 108 finished with value: 2.5246730514881923 and parameters: {'n_estimators': 450, 'max_features': 'log2', 'max_depth': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': True, 'criterion': 'mae'}. Best is trial 87 with value: 2.300202263425772.
[I 2022-11-25 03:18:28,550] Trial 109 finished with value: 2.4781645060807715 and parameters: {'n_estimators': 450, 'max_features': 'sqrt', 'max_depth': 100, 'min_samples_split': 6, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mae'}. Best is trial 87 with value: 2.300202263425772.
[I 2022-11-25 03:23:02,524] Trial 110 finished with value: 3.085865939681062 and parameters: {'n_estimators': 450, 'max_features': 'sqrt', 'max_depth': 100, 'min_samples_split': 2, 'min_samples_leaf': 4, 'bootstrap': False, 'criterion': 'mae'}. Best is trial 87 with value: 2.300202263425772.
[I 2022-11-25 03:28:24,299] Trial 111 finished with value: 2.300372928947902 and parameters: {'n_estimators': 450, 'max_

[I 2022-11-25 05:27:10,517] Trial 135 finished with value: 2.300372928947902 and parameters: {'n_estimators': 450, 'max_features': 'sqrt', 'max_depth': 80, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mae'}. Best is trial 87 with value: 2.300202263425772.
[I 2022-11-25 05:32:18,356] Trial 136 finished with value: 2.3150357528302696 and parameters: {'n_estimators': 450, 'max_features': 'sqrt', 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mae'}. Best is trial 87 with value: 2.300202263425772.
[I 2022-11-25 05:38:02,723] Trial 137 finished with value: 2.301586031304213 and parameters: {'n_estimators': 500, 'max_features': 'sqrt', 'max_depth': 80, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mae'}. Best is trial 87 with value: 2.300202263425772.
[I 2022-11-25 05:43:11,629] Trial 138 finished with value: 2.300372928947902 and parameters: {'n_estimators': 450, 'max_fea

Number of finished trials: 160
Best trial: {'n_estimators': 350, 'max_features': 'sqrt', 'max_depth': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False, 'criterion': 'mae'}
best rmse is : [2.300202263425772]


In [8]:
params = study.best_trial.params

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

loss_history = []
accuracy = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = ExtraTreesRegressor(random_state=42,**params) 
    model.fit(X_train, y_train) 

    y_pred = model.predict(X_test) 
    loss_history.append(mean_squared_error(y_test, y_pred,squared=False))
    accuracy.append(r2_score(y_test,y_pred,))
print("history of rmse :", loss_history)
print("average rmse value  :", np.mean(loss_history))
print("history of r2 square :",accuracy)
print("averare r2 square :" ,np.mean(accuracy))

history of rmse : [2.691860221836978, 2.0528577130296934, 2.488042971793109, 2.2052567190146815, 2.5056365719580276, 2.844860183217136, 2.1728016686280727, 2.0794624048396297, 2.088701170362757, 1.8725430095776363]
average rmse value  : 2.300202263425772
history of r2 square : [0.8462433552791554, 0.8754532603117453, 0.8732330220376744, 0.8784547538555179, 0.9118431760958569, 0.8745304311552667, 0.8394990119215334, 0.9107017680233, 0.8844611360340151, 0.9004653033261838]
averare r2 square : 0.8794885218040248
